In [3]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from sklearn.model_selection import train_test_split


tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.13.1


In [182]:
from pymystem3 import Mystem
from tqdm import tqdm_notebook,tqdm_pandas,tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import KeyedVectors

import nltk
import json
from pathlib import Path

import tf_metrics
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences


from multiprocessing import cpu_count, Pool
import gc

In [5]:
tqdm_pandas(tqdm)

In [12]:
with open('pikabu.csv') as f:
    print(f.read(500))
       

context,answer
 Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец.,Спасибо.
Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец. Спасибо.,Приедь к нам в Мурманск пожалуйста.
Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец. Спасибо.,Тебе платят за это?
Ты просто большой молодец. Спасибо. Тебе платят за это?,"За посты на Пикабу? Да, платят. Достаточно при написании поста всего лишь .."
"Спасибо. Тебе платят за это? За посты на Пикабу? Да, пл


In [6]:
df_train = pd.read_csv('pikabu.csv',sep=',')

In [7]:
df_train.head(20)

,context,answer
0,Убрал 600 литров мусора в реликтовом лесу. Ты...,Спасибо.
1,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Приедь к нам в Мурманск пожалуйста.
2,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Тебе платят за это?
3,Ты просто большой молодец. Спасибо. Тебе платя...,"За посты на Пикабу? Да, платят. Достаточно при..."
4,Спасибо. Тебе платят за это? За посты на Пикаб...,Не нажимается сук
5,Ты просто большой молодец. Спасибо. Тебе платя...,Лично Путин
6,Спасибо. Тебе платят за это? Лично Путин,дарт вейдар
7,Убрал 600 литров мусора в реликтовом лесу. Ты...,"ну всё, теперь можно не убирать за собой, Чист..."
8,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Зачем заминусили человека? Очевидная ирония же...
9,Убрал 600 литров мусора в реликтовом лесу.,"Герой, про которого я бы читал комиксы от dc и..."


In [14]:
df_train.count()

context    21486165
answer     21486162
dtype: int64

In [22]:
len(df_train.loc[df_train['context'].apply(len)>410,'context'][271713].split())

67

In [39]:
context.shape

(21486165,)

In [60]:
context[:20]

array([' Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец.',
       'Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец. Спасибо.',
       'Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец. Спасибо.',
       'Ты просто большой молодец. Спасибо. Тебе платят за это?',
       'Спасибо. Тебе платят за это? За посты на Пикабу? Да, платят. Достаточно при написании поста всего лишь ..',
       'Ты просто большой молодец. Спасибо. Тебе платят за это?',
       'Спасибо. Тебе платят за это? Лично Путин',
       ' Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец.',
       'Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец. ну всё, теперь можно не убирать за собой, Чистомен спасёт!',
       '  Убрал 600 литров мусора в реликтовом лесу.',
       ' Убрал 600 литров мусора в реликтовом лесу. Герой, про которого я бы читал комиксы от dc или марвел!',
       'Убрал 600 литров мусора в реликтовом лесу. Герой

In [107]:
k = np.array(list(map(text_prep_tags,tqdm_notebook(context[:20]))))


KeyError: 'убирать_VERB'

In [52]:
cpu_count()

4

In [109]:
id_text_vocab

{4: 'убирать_VERB'}

In [28]:
from nltk.corpus import stopwords
sw_ru = set(stopwords.words('russian'))

In [55]:
mystem = Mystem()

In [ ]:
context = df_train['context'].values

In [43]:
def index_tokenizer(tokenizer, corpus):
    #voc_set = set()
    res = np.fromiter(map(tokenizer,tqdm_notebook(corpus[:20]),),dtype=str)
    voc_set = np.unique(res)
    print(voc_set)

In [42]:
def fu(x):
    return np.apply_along_axis(text_prep_tags,1,np.reshape(x,(-1,1)))
        
def parallelize(data,partitions):

    data_split = np.array_split(data, partitions)
    pool = Pool(partitions)
    #data = np.stack(pool.apply(np.apply_along_axis,text_prep_tags,1,np.reshape(data_split,(-1,1))))
    data = np.concatenate(pool.map(fu,data_split))
    pool.close()
    pool.join()
    return data

ValueError: not enough values to unpack (expected 3, got 2)

In [56]:
%time np_context = parallelize(context[:20000],4)

CPU times: user 173 ms, sys: 342 ms, total: 515 ms
Wall time: 13.2 s


In [48]:
os.path()

In [140]:
class Vocab:
    def __init__(self):
        
        self.char2idx = {'<PAD>':0,'<START>':1,'<END>':2,'<UNK>':3}
#         self.char2idx.update({ch:i+4 for i,ch in enumerate(set(tokenize_string(data)))})
        self.idx2char = {i:ch for ch,i in self.char2idx.items()}
    
    def save(path='.'):
        json_ch_idx = json.dumps(voc.char2idx)
        with open(os.path.join(path,"char2idx.json"),"w") as f:
            f.write(json_ch_idx)
        json_idx_ch = json.dumps(voc.idx2char)
        with open(os.path.join(path,"idx2char.json"),"w") as f:
            f.write(json_idx_ch)
    
    def load(path='.'):
        
    
    def indx_tokenize(self, sequence):
        sequence = tokenize_string(sequence)
        return [self.char2idx[char] for char in sequence]
    
    def indx_detokenize(self, sequence):
        return ''.join([self.idx2char[idx] for idx in sequence])
    
    def tokenize(self, data, tokenizer, huge_parts, save_to_file=True):
        huge_splits = np.array_split(data, huge_parts)
        partitions = cpu_count()
        for split_number,huge_split in enumerate(tqdm_notebook(huge_splits)):
            print(huge_split.shape)          
            data_split = np.array_split(huge_split, partitions)
            pool = Pool(partitions)

            res_data = np.concatenate(pool.map(tokenizer,data_split))
            pool.close()
            pool.join()
            
            print(res_data.shape)
            self._extend_vocab(res_data)
            
            print(res_data)
            
            if save_to_file:
                np.savez_compressed(f'tokenized_files/np_tokens{split_number}',arr=res_data)             
                del res_data
                gc.collect()
                
                
    def _extend_vocab(self,token_matrix):
        start_idx = len(self.char2idx)
        for idx,token in np.ndenumerate(np.unique(token_matrix)):
            if token not in self.char2idx:
                self.char2idx[token] = start_idx + idx[0]
        
        self.idx2char = {i:ch for ch,i in self.char2idx.items()}
                
        
    
    def __len__(self):
        return len(self.char2idx)

In [166]:
def fu(x):
    return np.apply_along_axis(text_prep_tags,1,np.reshape(x,(-1,1)))
voc = Vocab()
%time voc.tokenize(context[:2000000],fu,2)

(1000000,)
(1000000, 40)
[['убирать_VERB' '600_UNKN' 'литр_NOUN' ... '' '' '']
 ['убирать_VERB' '600_UNKN' 'литр_NOUN' ... '' '' '']
 ['убирать_VERB' '600_UNKN' 'литр_NOUN' ... '' '' '']
 ...
 ['гейб_NOUN' 'молодец_NOUN' 'давно_ADV' ... '' '' '']
 ['гейб_NOUN' 'молодец_NOUN' 'давно_ADV' ... '' '' '']
 ['гейб_NOUN' 'молодец_NOUN' 'давно_ADV' ... '' '' '']]
(1000000,)
(1000000, 40)
[['молодец_NOUN' 'самый_DET' 'начало_NOUN' ... '' '' '']
 ['это_PRON' 'человек_NOUN' 'виноватый_ADJ' ... '' '' '']
 ['беда_NOUN' 'гринлайт_NOUN' 'плюсонуть_VERB' ... '' '' '']
 ...
 ['сложно_ADV' 'олдфаг_NOUN' 'зарегистрировать_VERB' ... '' '' '']
 ['зарегистрировать_VERB' 'месяц_NOUN' 'олдфаг_NOUN' ... '' '' '']
 ['это_PRON' 'читать_VERB' 'сидеть_VERB' ... '' '' '']]
CPU times: user 1min 33s, sys: 1min 7s, total: 2min 40s
Wall time: 18min 59s


In [174]:
voc.idx2char

{0: '<PAD>',
 1: '<START>',
 2: '<END>',
 3: '<UNK>',
 4: '',
 5: '    _ _UNKN',
 6: '    __   _UNKN',
 7: '   _ _UNKN',
 8: '   __\n_UNKN',
 9: '   __ __UNKN',
 10: '  _\n_UNKN',
 11: '  _  _UNKN',
 12: '  _ _\n_UNKN',
 13: '  _ _UNKN',
 14: '  __\n_UNKN',
 15: '  __  _UNKN',
 16: '  __  __UNKN',
 17: '  __ _UNKN',
 18: '  __UNKN',
 19: '  ___\n_UNKN',
 20: '  ___UNKN',
 21: '  ヽ\n_UNKN',
 22: '  ヽ _UNKN',
 23: '  ヽ ー__UNKN',
 24: '  ヽ_UNKN',
 25: ' _\n_UNKN',
 26: ' _   _UNKN',
 27: ' _  _UNKN',
 28: ' _ _ _\n_UNKN',
 29: ' _ _ _ _ _ _\n_UNKN',
 30: ' _ _ _ _ _ _ _ _ _ _ _ _ ',
 31: ' _ _ _ _ _ _ _UNKN',
 32: ' _ _ _ _UNKN',
 33: ' _ _ _UNKN',
 34: ' _ _UNKN',
 35: ' _ __UNKN',
 36: ' _ ヽ _UNKN',
 37: ' __\n_UNKN',
 38: ' __  _UNKN',
 39: ' __ _ _\n_UNKN',
 40: ' __ _UNKN',
 41: ' __ __UNKN',
 42: ' __UNKN',
 43: ' ___\n_UNKN',
 44: ' ___ _UNKN',
 45: ' ___UNKN',
 46: ' ____\n_UNKN',
 47: ' ____ _UNKN',
 48: ' ____UNKN',
 49: ' _____\n_UNKN',
 50: ' _____ _ _UNKN',
 51: ' _____ _UNKN

In [177]:
np_arr = None
for i in range(2):
    with np.load(f'tokenized_files/np_tokens{i}.npz') as data:
        if np_arr is not None:
            np_arr = np.append(np_arr,data['arr'],axis=0)
        else:
            np_arr = data['arr']


In [178]:
np_arr.shape

(2000000, 40)

In [179]:
np_arr.nbytes/1000000

8000.0

In [180]:
np_arr

array([['убирать_VERB', '600_UNKN', 'литр_NOUN', ..., '', '', ''],
       ['убирать_VERB', '600_UNKN', 'литр_NOUN', ..., '', '', ''],
       ['убирать_VERB', '600_UNKN', 'литр_NOUN', ..., '', '', ''],
       ...,
       ['сложно_ADV', 'олдфаг_NOUN', 'зарегистрировать_VERB', ..., '',
        '', ''],
       ['зарегистрировать_VERB', 'месяц_NOUN', 'олдфаг_NOUN', ..., '',
        '', ''],
       ['это_PRON', 'читать_VERB', 'сидеть_VERB', ..., '', '', '']],
      dtype='<U25')

In [63]:
%time np.unique(np_context)[100]

CPU times: user 264 ms, sys: 10.4 ms, total: 274 ms
Wall time: 260 ms


'139_UNKN'

In [176]:
import json
 
json1 = json.dumps(voc.idx2char)
f = open("idx2char.json","w")
f.write(json1)
f.close()
json2 = json.dumps(voc.char2idx)
f = open("char2idx.json","w")
f.write(json2)
f.close()

In [74]:
for idx,token in np.ndenumerate(np.unique(np_context[:2])):
    print(idx,token)

(0,) 
(1,) 600_UNKN
(2,) большой_ADJ
(3,) лес_NOUN
(4,) литр_NOUN
(5,) молодец_NOUN
(6,) мусор_NOUN
(7,) просто_PART
(8,) реликтовый_ADJ
(9,) спасибо_NOUN
(10,) убирать_VERB


In [50]:
del np_context
gc.collect()

NameError: name 'np_context' is not defined

In [49]:
r = np.array([3,4,3,6])
r.shape

(4,)

In [65]:
y = np.empty([400], dtype="str")
y[:r.shape[0]] = r
y

array(['3', '4', '3', '6', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', ''

In [117]:
text_id_vocab = {}
id_text_vocab = {}
prev_word = None

def text_prep_tags(text):
   
    #print(text)
    text = text[0]
    upos_map = {'A':'ADJ','ADV':'ADV','ADVPRO':'ADV','ANUM':'ADJ','APRO':'DET','COM':'ADJ','CONJ':'SCON','INTJ':'INTJ','NONLEX':'X','NUM':'NUM','PART':'PART','PR':'ADP','S':'NOUN','SPRO':'PRON','UNKN':'X' ,'V':'VERB'}
    text = text.lower()
    result = np.array([])
    
    
    # Убираем лишние символы
    #text = re.sub(r'[;,]',r' ',text).strip()
    text = re.sub(r'[^\w\s\.]',r'',text).strip()   
    #text = [token.text for token in razdel.tokenize(text)]
    # Делаем лемматизацию       
#     text = [lemma for lemma in mystem.lemmatize(text) if not lemma.isspace() and lemma not in sw_ru
#             and lemma.strip() not in ['.','..','...']]
    
#     if id_text_vocab != {}:
#         prev_word = id_text_vocab[max(id_text_vocab.keys())]
#     else:       
#         prev_word = None
        
    for item in mystem.analyze(text):
      #  print(item)
        token = None
        if item.get('analysis'):
            lemma = item['analysis'][0]['lex']
            pos = re.split('[=,]', item['analysis'][0]['gr'])[0]
            #and lemma not in sw_ru
            if not lemma.isspace()  and lemma.strip() not in ['.','..','...'] and lemma not in sw_ru:
     
                token = f'{lemma}_{upos_map[pos]}'
        else:
            lem_text = item["text"]
            if not lem_text.isspace() and lem_text.strip() not in ['.','..','...'] and lem_text not in sw_ru:
            
                token = f'{lem_text}_UNKN'
            
        if token:    
            #result.append(token)
            result = np.append(result,token)
            
#             if prev_word:
#                 text_id_vocab[token] = text_id_vocab[prev_word] + 1
#                 id_text_vocab[text_id_vocab[prev_word] + 1] = token
#             else:
#                 text_vocab[token] = 4
#                 id_text_vocab[4] = token
#             prev_word = token

    
    
    
    # Лемматизатор неправильно разбивает слова с дефисом, поэтому исправляем это
#     if '-' in text:
#         for l in range(len(text)):
#             if text[l] == '-':
#                 text[l] = f'{text[l-1]}-{text[l+1]}'
#                 text[l-1] = text[l+1] = text[l]
    zer = np.empty([40], dtype="U25")
    size = result.shape[0]
    if size>40:
        zer = result[:40]
    else:
        zer[:size] = result
        
    return zer

In [ ]:
word2vec = KeyedVectors.load_word2vec_format('../../model.bin', binary=True)

In [82]:
train_token_text = df_train.loc[:100000,'context'].progress_apply(text_prep_tags)
#test_token_text = df_test['text'].progress_apply(text_prep_tags)

100%|██████████| 100001/100001 [02:23<00:00, 698.76it/s]


In [84]:
train_token_text = df_train['context'].progress_apply(text_prep_tags)

  1%|          | 148710/21486165 [03:47<8:19:13, 712.36it/s] 

KeyboardInterrupt: 

In [ ]:
count_v = CountVectorizer(tokenizer=text_prep_tags)
count_v.fit(tqdm_notebook(pd.concat([df_train['text'],df_test['text']])))

In [80]:
train_token_text

0         [None, None, None, None, None, None, None, Non...
1         [None, None, None, None, None, None, None, Non...
2         [None, None, None, None, None, None, None, Non...
3         [т_NOUN, None, None, None, None, None, None, N...
4         [None, None, None, None, None, None, None, Non...
5         [т_NOUN, None, None, None, None, None, None, N...
6         [None, None, None, None, None, None, None, Non...
7         [None, None, None, None, None, None, None, Non...
8         [None, None, None, None, None, None, None, Non...
9         [None, None, None, None, None, None, None, Non...
10        [None, None, None, None, None, None, None, Non...
11        [None, None, None, None, None, None, None, Non...
12        [г_NOUN, None, None, None, None, None, None, N...
13        [д_NOUN, None, None, None, None, None, None, N...
14        [г_NOUN, None, None, None, None, None, None, N...
15        [None, None, None, None, None, None, None, Non...
16        [г_NOUN, None, None, None, Non